In [1]:
import $file.^.init.spark, spark._
import $file.^.init.paths, paths._
import $file.^.init.glow, glow._
import io.projectglow.Glow

val ss = getLocalSparkSession()
import ss.implicits._
Glow.register(ss)

Compiling /home/eczech/repos/gwas-analysis/notebooks/tutorial/init/spark.scCompiling /home/eczech/repos/gwas-analysis/notebooks/tutorial/init/paths.scCompiling /home/eczech/repos/gwas-analysis/notebooks/tutorial/init/glow.scLoading spark-stubs


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/eczech/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.16/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/eczech/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.25/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/01/16 12:37:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $file.$           , spark._

import $file.$           , paths._

import $file.$          , glow._

import io.projectglow.Glow


ss: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@34ae8466
import ss.implicits._


In [2]:
val data_dir = GWAS_TUTORIAL_DATA_DIR / "1_QC_GWAS"

data_dir: better.files.File = /home/eczech/data/gwas/tutorial/1_QC_GWAS

## Load and Cache

In [3]:
val df = ss.read.format("plink").load(data_dir / "HapMap_3_r3_1.bed" toString)

df: org.apache.spark.sql.package.DataFrame = [contigName: string, names: array<string> ... 6 more fields]

In [4]:
df.printSchema

root
 |-- contigName: string (nullable = true)
 |-- names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- position: double (nullable = true)
 |-- start: long (nullable = true)
 |-- end: long (nullable = true)
 |-- referenceAllele: string (nullable = true)
 |-- alternateAlleles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genotypes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- sampleId: string (nullable = true)
 |    |    |-- calls: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [13]:
df.write.format("parquet").save(data_dir / "HapMap_3_r3_1.parquet" toString)

save at cmd12.sc:1

15 / 15

## Reload

In [14]:
val df = ss.read.parquet(data_dir / "HapMap_3_r3_1.parquet" toString)

parquet at cmd13.sc:1

1 / 1

df: DataFrame = [contigName: string, names: array<string> ... 6 more fields]

In [15]:
df.count

count at cmd14.sc:1

15 / 15

count at cmd14.sc:1

1 / 1

res14: Long = 1457897L

In [16]:
df.groupBy(size($"genotypes.calls")).count.show

show at cmd15.sc:1

15 / 15

show at cmd15.sc:1

1 / 1

+---------------------+-------+
|size(genotypes.calls)|  count|
+---------------------+-------+
|                  165|1457897|
+---------------------+-------+



In [16]:
df
    .withColumn("genotypes", explode($"genotypes"))
    .show(3, false)

show at cmd15.sc:3

1 / 1

+----------+-----------+--------+------+------+---------------+----------------+----------------------+
|contigName|names      |position|start |end   |referenceAllele|alternateAlleles|genotypes             |
+----------+-----------+--------+------+------+---------------+----------------+----------------------+
|1         |[rs2185539]|0.0     |556737|556738|C              |[T]             |[1328_NA06989, [0, 0]]|
|1         |[rs2185539]|0.0     |556737|556738|C              |[T]             |[1377_NA11891, [0, 0]]|
|1         |[rs2185539]|0.0     |556737|556738|C              |[T]             |[1349_NA11843, [0, 0]]|
+----------+-----------+--------+------+------+---------------+----------------+----------------------+
only showing top 3 rows



In [18]:
// Show number of calls per sample + variant combo
// Should be across num variants * num samples data points
print(s"Expected: ${1457897 * 165}")
df
    .withColumn("genotypes", explode($"genotypes"))
    .groupBy(size($"genotypes.calls")).count.show

Expected: 240553005

show at cmd17.sc:4

15 / 15

show at cmd17.sc:4

1 / 1

+---------------------+---------+
|size(genotypes.calls)|    count|
+---------------------+---------+
|                    2|240553005|
+---------------------+---------+



In [19]:
df
    .withColumn("genotypes", explode($"genotypes"))
    .groupBy(size($"names"), size($"alternateAlleles")).count.show

show at cmd18.sc:3

15 / 15

show at cmd18.sc:3

1 / 1

+-----------+----------------------+---------+
|size(names)|size(alternateAlleles)|    count|
+-----------+----------------------+---------+
|          1|                     1|240553005|
+-----------+----------------------+---------+



#### Call Rate

Calculate variant call rates (i.e. across samples) manually and compare to ```call_summary_stats```.

In [20]:
df
    .withColumn("genotypes", explode($"genotypes"))
    .withColumn("call", explode_outer($"genotypes.calls"))
    .withColumn("sampleId", $"genotypes.sampleId")
    .drop("genotypes")
    .show(3, false)

show at cmd19.sc:6

1 / 1

+----------+-----------+--------+---------+---------+---------------+----------------+----+------------+
|contigName|names      |position|start    |end      |referenceAllele|alternateAlleles|call|sampleId    |
+----------+-----------+--------+---------+---------+---------------+----------------+----+------------+
|6         |[rs1890312]|0.0     |153585564|153585565|G              |[A]             |0   |1328_NA06989|
|6         |[rs1890312]|0.0     |153585564|153585565|G              |[A]             |0   |1328_NA06989|
|6         |[rs1890312]|0.0     |153585564|153585565|G              |[A]             |0   |1377_NA11891|
+----------+-----------+--------+---------+---------+---------------+----------------+----+------------+
only showing top 3 rows



In [5]:
// See https://glow.readthedocs.io/en/latest/etl/utility-functions.html
def dfc = df
    .selectExpr("*", "genotype_states(genotypes) as states")
    .withColumn("variantId", $"names"(0))
    .withColumn("sampleId", $"genotypes.sampleId")
    .selectExpr(
        "variantId", 
        "explode(arrays_zip(sampleId, states)) as genotypes"
    )
    .select("variantId", "genotypes.*")
dfc.printSchema

root
 |-- variantId: string (nullable = true)
 |-- sampleId: string (nullable = true)
 |-- states: integer (nullable = true)



defined function dfc

In [6]:
// Frequency of call types showing that homozygous reference is most common by far
dfc.groupBy("states").count.show

20/01/16 12:38:36 INFO PlinkFileFormat$: hlsUsage:[plinkRead,{"includeSampleIds":true,"mergeFidIid":true}]
20/01/16 12:38:41 INFO PlinkFileFormat: Reading variants [99865, 199729]
20/01/16 12:38:41 INFO PlinkFileFormat: Reading variants [199729, 299594]
20/01/16 12:38:41 INFO PlinkFileFormat: Reading variants [299594, 399458]
20/01/16 12:38:41 INFO PlinkFileFormat: Reading variants [0, 99865]
20/01/16 12:38:41 INFO PlinkFileFormat: Reading variants [599187, 699051]
20/01/16 12:38:41 INFO PlinkFileFormat: Reading variants [798915, 898780]
20/01/16 12:38:41 INFO PlinkFileFormat: Reading variants [499322, 599187]
20/01/16 12:38:41 INFO PlinkFileFormat: Reading variants [399458, 499322]
20/01/16 12:38:41 INFO PlinkFileFormat: Reading variants [898780, 998644]
20/01/16 12:38:41 INFO PlinkFileFormat: Reading variants [1098509, 1198373]
20/01/16 12:38:41 INFO PlinkFileFormat: Reading variants [1398102, 1457897]
20/01/16 12:38:41 INFO PlinkFileFormat: Reading variants [998644, 1098509]
20/01/1

+------+---------+
|states|    count|
+------+---------+
|     0|158635633|
|     2| 15818420|
|    -1|   630620|
|     1| 65468332|
+------+---------+



In [40]:
def dfcv = dfc
    .groupBy("variantId").agg(mean(($"states" >= 0).cast("float")).as("callRate"))
dfcv.show(3)

show at cmd39.sc:3

15 / 15

show at cmd39.sc:3

1 / 1

+----------+--------+
| variantId|callRate|
+----------+--------+
| rs1890312|     1.0|
|  rs317105|     1.0|
|rs10499272|     1.0|
+----------+--------+
only showing top 3 rows



defined function dfcv

In [43]:
def dfcg = df
    .selectExpr(
        "element_at(names, 1) as variantId", 
        "call_summary_stats(genotypes).callRate as callRate"
    )
dfcg.show(3) 

show at cmd42.sc:6

1 / 1

+----------+--------+
| variantId|callRate|
+----------+--------+
| rs1890312|     1.0|
|  rs317105|     1.0|
|rs10499272|     1.0|
+----------+--------+
only showing top 3 rows



defined function dfcg

In [48]:
def dfj = dfcv
    .withColumnRenamed("callRate", "callRateManual")
    .join(dfcg.withColumnRenamed("callRate", "callRateFunction"), Seq("variantId"), "outer")
dfj.printSchema

root
 |-- variantId: string (nullable = true)
 |-- callRateManual: double (nullable = true)
 |-- callRateFunction: double (nullable = true)



defined function dfj

In [49]:
dfj.groupBy($"callRateManual" === $"callRateFunction").count.show

show at cmd48.sc:1

15 / 15

show at cmd48.sc:1

15 / 15

show at cmd48.sc:1

1 / 1

show at cmd48.sc:1

1 / 1

+-----------------------------------+-------+
|(callRateManual = callRateFunction)|  count|
+-----------------------------------+-------+
|                               true|1457897|
+-----------------------------------+-------+



## Allele Frequencies

In [55]:
def dfaf = df
    .selectExpr(
        "element_at(names, 1) as variantId", 
        "call_summary_stats(genotypes) as stats"
    )
dfaf.printSchema

root
 |-- variantId: string (nullable = true)
 |-- stats: struct (nullable = true)
 |    |-- callRate: double (nullable = false)
 |    |-- nCalled: integer (nullable = false)
 |    |-- nUncalled: integer (nullable = false)
 |    |-- nHet: integer (nullable = false)
 |    |-- nHomozygous: array (nullable = true)
 |    |    |-- element: integer (containsNull = false)
 |    |-- nNonRef: integer (nullable = false)
 |    |-- nAllelesCalled: integer (nullable = false)
 |    |-- alleleCounts: array (nullable = true)
 |    |    |-- element: integer (containsNull = false)
 |    |-- alleleFrequencies: array (nullable = true)
 |    |    |-- element: double (containsNull = false)



defined function dfaf

In [58]:
dfaf.select("stats.nAllelesCalled", "stats.alleleCounts", "stats.alleleFrequencies").show(3, false)

show at cmd57.sc:1

1 / 1

+--------------+------------+-----------------------------------------+
|nAllelesCalled|alleleCounts|alleleFrequencies                        |
+--------------+------------+-----------------------------------------+
|330           |[330]       |[1.0]                                    |
|330           |[178, 152]  |[0.5393939393939394, 0.46060606060606063]|
|330           |[330]       |[1.0]                                    |
+--------------+------------+-----------------------------------------+
only showing top 3 rows



In [60]:
df.printSchema

root
 |-- contigName: string (nullable = true)
 |-- names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- position: double (nullable = true)
 |-- start: long (nullable = true)
 |-- end: long (nullable = true)
 |-- referenceAllele: string (nullable = true)
 |-- alternateAlleles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genotypes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- sampleId: string (nullable = true)
 |    |    |-- calls: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)

